In [2]:
import numpy as np

## ReLU layer

In [3]:
class ReLU:
    def __init__(self):
        # 순전파 때 x 값이 음수였던 위치를 mask => 역전파 때 0으로 내보낼 수 있도록
        self.mask = None

    def forward(self, x):
        self.mask = x <= 0
        out = x.copy()

        out[self.mask] = 0  # numpy 에서 주로 사용하는 boolean mask indexing
        return out

    def backward(self, d_out):
        """
        d_out : 뒤에서부터 전달된 미분값이 들어있는 배열이 들어온다
        """
        d_out[self.mask] = 0  # 순전파 때 음수였던 부분을 0으로 만들어준다

        d_x = d_out
        return d_x

In [4]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])

relu = ReLU()
relu.forward(x)

array([[1., 0.],
       [0., 3.]])

In [5]:
d_out = np.array([[-1.0, 0.5], [-2.0, 3.0]])

relu.backward(d_out)

array([[-1.,  0.],
       [ 0.,  3.]])

# $\sigma$(시그모이드) 구현
$$
\sigma(x) = \frac{\mathrm{1} }{\mathrm{1} + exp(-x)}
$$

$$
y = \sigma(x)
$$

$$
y' = \sigma(x)(1-\sigma(x))
$$

$$
y' = y(1-y)
$$

* 시그모이드 레이어에서 순전파 때 기억하고 있어야 할 값 : $y$값만 알고 있으면 된다.

In [6]:
class Sigmoid:
    def __init__(self):
        # 순전파 때 구한 y로 역전파 수행 시 y(1-y) 계산
        self.out = None

    def forward(self):
        out = 1 / (1 + np.exp(-x))
        self.out = out  # 역전파에 사용하기 위해 저장

        return out

    def backward(self, d_out):
        d_x = d_out * self.out * (1 - self.out)
        return d_x

# Linear(Dense, FCL) 계층
* `forward`
  1. 입력값 `x`와 가중치 `W`의 내적 + `b` (`WX+b`)
  2. 평탄화
    * 1차원 배열로 데이터가 들어왔을 때 대응
    * 다차원 배열( 텐서 )에 대한 대응
    * **원본 데이터의 형상을 저장**
* `backward`
  1. 미분값(`dout`)과 가중치의 전치행렬(`W.T`) 내적 ( 입력값에 대한 미분값 )
  2. 입력값의 전치행렬(`X.T`)과 미분값(`dout`) 내적 ( 가중치에 대한 미분값 )
  3. 배치를 축으로 편향 합 구하기 (`axis=0`)
  4. 입력값의 미분값(`dx`)의 형상을 원본 `x`의 형상으로 다시 바꿔주기

`forward`에 의해서 형상(shape) 변환이 일어나기 때문에 `backward`할 때 원본 모양으로 되돌려 준다.
* `(100, 28, 28)`이 입력으로 들어오면, `forward`에서 평탄화에 의해 `(100, 784)`가 된다.
* 텐서에도 모두 내적을 수행할 수 있도록 원본 형상인 `(100, 28, 28)`저장을 해 놨다가 `backward`할 때 저장했던 원본 모양으로 **미분값 배열**의 형상을 되돌려 준다.

**저장해야 할 값**
1. 원본 `x`의 형상(shape)
2. 원본 `x` 데이터
3. 가중치, 편향
4. `dW`, `db`
  * 미분값을 알고 있어야 나중에 최적화( Optimization )를 할 수 있다.

In [7]:
class Linear:
    def __init__(self, W, b):
        # 가중치, 편향, 입력값
        self.W = W
        self.b = b
        self.x = None

        # x 의 shape 저장 : 지금은 평탄화 된 데이터를 사용하기 때문에 필수는 아니지만, 나중에 CNN 등 데이터가 1차원이 아닐 때 필요
        self.original_x_shape = None

        # W 와 b의 기울기 배열 저장 => Optimization 을 위해 각 매개변수의 미분값이 필요
        self.d_W = None
        self.d_b = None

    def forward(self, x):
        # 텐서 대응을 위해 입렦밧 x의 shape 저장
        # ex) x 가 (3, 2, 2) => 선형연산을 위해서는 (3, 2*2)로 평탄화 필요 => 역전파시 다시 (3, 2, 2)로 바꿔줘야 함
        self.original_x_shape = x.shape

        # 평탄화
        DATA_SIZE = x.shape[0]  # batch size
        x = x.reshape(DATA_SIZE, -1)

        self.x = x
        out = self.x @ self.W + self.b  # (N, M) @ (M, K) + (K, ) => (N, K)

        return out

    def backward(self, d_out):  # d_out 의 shape : (N, K)
        # 입력값에 내보내는 미분값 계산
        d_x = d_out @ self.W.T  # (N, K) @ (K, M) => (N, M)

        self.d_W = self.x.T @ d_out  # (M, N) @ (N, K) => (M, K)
        # (K, ) 였던 bias 가 순전파를 거치면서 batch size 만큼 증폭됨 (batch 마다 더해져서)
        # 역전파에서 다시 바꿔줘야 한다
        self.d_b = np.sum(d_out, axis=0)  # (N, K) => (K, )

        return d_x

In [8]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T
    x = x - np.max(x)  # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [9]:
class SoftmaxWithLoss:

    def __init__(self):
        self.loss = None  # 손실값 (시각화 할 때 사용할 예정..)
        self.y = None  # 예측값 ( 역전파 때 사용 )
        self.t = None  # 정답 레이블 ( 역전파 때 사용 )

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, d_out=1):  # dout이 1인 이유 : d돈통 / d포스기 개념
        # 배치 고려하기
        batch_size = self.t.shape[0]

        # t가 원-핫 인코딩이 되어있는지, 안되어 있는지 고려
        if (
            self.t.size == self.y.size
        ):  # 출력층의 원소 개수를 비교하는 것은 원-핫 인코딩이 되어있는 t
            # 항상 y는 softmax의 결과물 (N, OUTPUT_SIZE)
            # t가 OHE가 되어 있으면 (N, OUTPUT_SIZE)
            # t가 OHE가 안되어 있으면 (N, )
            d_x = (self.y - self.t) / batch_size
        else:  # t가 OHE가 안되어 있는 경우
            d_x = self.y.copy()

            # 원-핫 인코딩이 되어있지 않은 t는 정답 레이블의 인덱스로 생각할 수 있다.
            # y = [0.2, 0.1, 0.7], t = 2
            # dx[np.arange(batch_size), self.t] -> dx[0, 2] -> 0.7 -> 0.7 - 1 를 구하겠다는 이야기 이다. -> -0.3의 오차가 있다.
            d_x[np.arange(batch_size), self.t] -= 1
            d_x = d_x / batch_size

        return d_x

In [10]:
# OrderedDict : 순서가 있는 딕셔너리
from collections import OrderedDict


# 2층 신경망 구현
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 매개변수 초기화 (모델 파라미터 -> 미분이 가능해야 한다)
        self.params = {}

        # 1층 은닉층을 위한 파라미터
        self.params["W1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)

        # 2층 출력층을 위한 매개변수
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)

        # 레이어 정의
        # 추가되는 레이어의 순서가 바뀌면 안되기 때문에 OrderedDict 사용
        self.layers = OrderedDict()

        self.layers["linear_1"] = Linear(self.params["W1"], self.params["b1"])
        self.layers["relu"] = ReLU()

        self.layers["linear_2"] = Linear(self.params["W2"], self.params["b2"])

        # 마지막 층은 항상 SoftmaxWithLoss
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        # 각 층의 forward 함수를 순차적으로 실행
        for layer in self.layers.values():  # 레이어를 순서대로 가져오기
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t):
        # 예측을 하고, 정답이랑 얼마나 틀렸는지를 계산
        # 단, t가 원-핫 인코딩이 되어있는지, 안되어있는지가 중요!
        y = self.predict(x)
        y = np.argmax(
            y, axis=1
        )  # 제일 큰 값 하나만 뽑자.( 제일 확률이 높은 곳에 위치한 인덱스를 갖는다. )

        # 원-핫 인코딩 처리
        # t.ndim != 1 --> t가 원핫 인코딩이 되어있는 상태라면
        # t에서 제일 높은 인덱스를 찾겠다.
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])

        return accuracy

    # 역전파를 활용한 각 매개변수의 기울기 배열 구하기
    def backward_propagation(self, x, t):
        # 순전파를 통해 오차를 구한다
        self.loss(x, t)

        # 역전파
        d_out = 1  # dL/dL = 1

        # SoftmaxWithLoss의 오차에 대한 미분값 구하기
        d_out = self.last_layer.backward(d_out)  # 순수한 오차 y-t

        # 은닉충에 해당하는 레이어만 받아와서 뒤집기
        hidden_layers = list(self.layers.values())  # 순서대로 되어있는 레이어
        hidden_layers.reverse()

        # 뒤집어진 레이어를 하나씩 꺼내기
        for layer in hidden_layers:
            # 미분값을 집어넣으며 역전파
            d_out = layer.backward(d_out)

        # 역전파가 끝났으면 각 레이어마다 기울기(d_W, d_b)가 들어있다
        grads = {}

        grads["W1"], grads["b1"] = (
            self.layers["linear_1"].d_W,
            self.layers["linear_1"].d_b,
        )

        grads["W2"], grads["b2"] = (
            self.layers["linear_2"].d_W,
            self.layers["linear_2"].d_b,
        )

        return grads

In [11]:
from tensorflow.keras import datasets  # type: ignore
from sklearn.preprocessing import OneHotEncoder

mnist = datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

y_train_dummy = OneHotEncoder().fit_transform(y_train.reshape(-1, 1))
y_train_dummy = y_train_dummy.toarray()

y_test_dummy = OneHotEncoder().fit_transform(y_test.reshape(-1, 1))
y_test_dummy = y_test_dummy.toarray()

# feature 전처리
X_train = X_train.reshape(X_train.shape[0], -1)
X_train = (
    X_train / 255.0
)  # 이미지 정규화 기법. 255.0 으로 나눠주면 모든 픽셀 데이터가 0 ~ 1사이의 값을 갖게 되고, 훈련이 쉽게 된다.

X_test = X_test.reshape(X_test.shape[0], -1)
X_test = X_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [12]:
model = TwoLayerNet(input_size=28 * 28, hidden_size=100, output_size=10)

In [13]:
# 하이퍼 파라미터 설정
iter_nums = 10000  # 학습 반복 횟수
learning_rate = 0.1
batch_size = 100
train_size = X_train.shape[0]

# 훈련 과정을 1에폭마다 기록 (시각화 하기 위해)
train_loss_list = []
test_loss_list = []

train_acc_list = []
test_acc_list = []

In [14]:
# 1 에폭에 필요한 훈련 횟수
iter_per_epoch = int(max(train_size / batch_size, 1))

for i in range(iter_nums):
    # 미니 배치 생성
    batch_mask = np.random.choice(train_size, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]

    # 훈련 (경사하강법에 의한 가중치, 편향 갱신)
    grads = model.backward_propagation(X_batch, y_batch)  # 기울기 얻기

    # optimization
    for key in ["W1", "b1", "W2", "b2"]:
        model.params[key] -= learning_rate * grads[key]

    # 1에폭 마다 정확도, loss 확인
    if i % iter_per_epoch == 0:
        train_loss = model.loss(X_batch, y_batch)
        test_loss = model.loss(X_test, y_test)

        train_acc = model.accuracy(X_train, y_train)
        test_acc = model.accuracy(X_test, y_test)

        train_loss_list.append(train_loss)
        test_loss_list.append(test_loss)

        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print(
            "Epoch {} ==> Train Accuracy : {:.6f} / Test Accuracy : {:.6f} / Train Loss : {:.6f} / Test Loss : {:.6f} ".format(
                int(i / iter_per_epoch), train_acc, test_acc, train_loss, test_loss
            )
        )

Epoch 0 ==> Train Accuracy : 0.116483 / Test Accuracy : 0.117200 / Train Loss : 2.300136 / Test Loss : 2.302532 
Epoch 1 ==> Train Accuracy : 0.902117 / Test Accuracy : 0.906300 / Train Loss : 0.384542 / Test Loss : 0.313861 
Epoch 2 ==> Train Accuracy : 0.928083 / Test Accuracy : 0.927300 / Train Loss : 0.224303 / Test Loss : 0.244362 
Epoch 3 ==> Train Accuracy : 0.941517 / Test Accuracy : 0.939900 / Train Loss : 0.129607 / Test Loss : 0.207493 
Epoch 4 ==> Train Accuracy : 0.948483 / Test Accuracy : 0.945700 / Train Loss : 0.115185 / Test Loss : 0.182400 
Epoch 5 ==> Train Accuracy : 0.955400 / Test Accuracy : 0.951400 / Train Loss : 0.074273 / Test Loss : 0.162393 
Epoch 6 ==> Train Accuracy : 0.961617 / Test Accuracy : 0.957600 / Train Loss : 0.073201 / Test Loss : 0.141216 
Epoch 7 ==> Train Accuracy : 0.964700 / Test Accuracy : 0.960200 / Train Loss : 0.078577 / Test Loss : 0.132643 
Epoch 8 ==> Train Accuracy : 0.967733 / Test Accuracy : 0.962100 / Train Loss : 0.168717 / Test 